# Gestion des biblothèques

### Installation de pulp

In [ ]:
!pip install pulp

### Importation des bibliothèques

In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpMaximize
import pandas as pd
import ast

# Pré traitement des données

### Importation du dataset

In [ ]:
recipes_df = pd.read_csv("../data/raw/RAW_recipes.csv")

### Suppression des colonnes inutiles

In [ ]:
recipes_df = recipes_df.drop(columns=['id', 'submitted', 'contributor_id', 'tags', 'description', 'ingredients', 'steps'])
recipes_df.info()

### Suppression des lignes incorrectes

In [ ]:
recipes_df.drop_duplicates()
recipes_df.dropna(inplace=True)

### Découpage de la colonne nutrition

In [ ]:
recipes_df['nutrition'] = recipes_df['nutrition'].apply(ast.literal_eval)
nutrition_columns = recipes_df['nutrition'].apply(pd.Series)

# Renommer les colonnes
nutrition_columns.columns = ['calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']

# Ajouter les nouvelles colonnes au DataFrame original
recipes_df = pd.concat([recipes_df, nutrition_columns], axis=1)

# Suppression de la colonne nutrition
recipes_df = recipes_df.drop(columns=['nutrition'])

#### Transformation des flottants en entiers

In [ ]:
for column in nutrition_columns.columns:
  recipes_df[column] = recipes_df[column].astype(int)

# Programmation par contrainte

### Définition des préférences utilisateurs

In [ ]:
user_preferences = {
    "calories": (200, 400), # Plage souhaitée pour les calories
    "total_fat": (0, 30),   # Plage souhaitée pour les matières grasses
    "sugar": (0, 20),       # Plage souhaitée pour le sucre
    "sodium": (0, 5)        # Plage souhaitée pour le sel
}

criteria = ["calories", "total_fat", "sugar", "sodium"]

### Définitions des paramètres

In [ ]:
n = len(recipes_df)
max_total_calories = 1000
min_total_protein = 20

### Définition de la satisfaction

#### Création de la fonction

In [ ]:
def compute_satisfaction(row, preferences):
    satisfaction = 0

    for criterion, (min_val, max_val) in preferences.items():
        value = row[criterion]

        if min_val <= value <= max_val:
            satisfaction += 1  # Complètement dans la plage
        elif value < min_val:
            satisfaction += 1 - (min_val - value) / min_val  # En dessous de la plage
        elif value > max_val:
            satisfaction += 1 - (value - max_val) / max_val  # Au dessus de la plage

    return satisfaction

#### Calcul du score pour toutes les recettes

In [ ]:
recipes_df["satisfaction"] = recipes_df.apply(
    lambda row: compute_satisfaction(row, user_preferences), axis=1
)

recipes_df = recipes_df.reset_index(drop=True)

### Définition du problème

In [ ]:
prob = LpProblem("Maximize_User_Satisfaction", LpMaximize)

#### Définition de la variable

In [ ]:
X = [LpVariable(f"X{i}", cat="Binary") for i in range(n)]

#### Définition des contraintes

In [ ]:
# Maximiser la satisfaction totale
prob += lpSum(recipes_df["satisfaction"][i] * X[i] for i in range(n)), "Total_Satisfaction"

In [ ]:
# Contrainte sur la quantité globale de calories
prob += lpSum(recipes_df["calories"][i] * X[i] for i in range(n)) <= max_total_calories, "Max_Calories"

In [ ]:
# Contrainte sur la quantité globale de protéine
prob += lpSum(recipes_df["protein"][i] * X[i] for i in range(n)) >= 20, "Max_Protein"

In [ ]:
# Limiter aux meilleurs 25 recettes maximum
max_recipes = 25
prob += lpSum(X[i] for i in range(n)) <= max_recipes, "Max_Recipes"

### Résolution du problème

In [ ]:
status = prob.solve()

#### Affichage des résultats

In [ ]:
if status == 1:
    print("Solution optimale trouvée !")

    # Récupération des recette sélectionnées
    selected_recipes = [i for i in range(n) if X[i].value() == 1]

    # Calcul des taux en calories et protéine
    total_calories = sum(recipes_df["calories"][i] * X[i].value() for i in range(n))
    total_protein = sum(recipes_df["protein"][i] * X[i].value() for i in range(n))

    # Affichage dans la console
    print(f"Nombre de recettes sélectionnées : {len(selected_recipes)}")
    print(f"Calories totales : {total_calories}")
    print(f"Protéines totales : {total_protein}")

    print("Recettes proposées :")
    display(recipes_df.iloc[selected_recipes])

else:
    print("Pas de solution optimale trouvée.")